# Step 01: Execute Analysis

This notebook submits analysis jobs to Moody's Risk Modeler.

**Tasks:**
- Retrieve Analysis batch from Stage_01/Step_03
- Review analysis job configurations
- Submit analysis jobs to Moody's API
- Track job completion status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Execute_Analysis.ipynb')

# Display context
ux.header("Execute Analysis Batch")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Analysis Batch

In [ ]:
# Retrieve Analysis batch from Stage_01/Step_03
ux.subheader("Retrieve Analysis Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.ANALYSIS not in batches:
    raise ValueError(f"Analysis batch not found. Available: {list(batches.keys())}")

analysis_batch_id = int(batches[BatchType.ANALYSIS])

ux.success(f"Retrieved Analysis batch: ID={analysis_batch_id}")
step.log(f"Retrieved Analysis batch: ID={analysis_batch_id}")

## 3) Review Analysis Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(analysis_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(analysis_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nSample Analysis Configurations (first 10):")
    rows = []
    for i, job in enumerate(jobs[:10]):
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            
            # Truncate long profile names
            analysis_profile = config.get('Analysis Profile', 'N/A')
            if len(analysis_profile) > 30:
                analysis_profile = analysis_profile[:27] + '...'
            
            rows.append([
                config.get('Analysis Name', 'N/A'),
                config.get('Portfolio', 'N/A'),
                config.get('Database', 'N/A'),
                analysis_profile
            ])
    ux.table(rows, headers=["Analysis Name", "Portfolio", "EDM", "Analysis Profile"])
    
    if job_count > 10:
        ux.info(f"... and {job_count - 10} more analysis job(s)")

step.log(f"Verified batch: {job_count} jobs ready for submission")

## 4) Submit Analysis Batch to Moody's

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
ux.info("  - Each job will look up EDM and portfolio in Moody's")
ux.info("  - Analysis profiles, output profiles, and event rate schemes will be validated")
ux.info("  - Treaties and tags will be resolved by name")
ux.info("  - Jobs will transition to SUBMITTED status")
ux.info("  - Batch will transition to ACTIVE status")
ux.info("")

# Submit
ux.info("\nSubmitting batch...")

# Pass step.step_id to associate batch with this step (not the creation step)
result = submit_batch(analysis_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\nBatch submission completed")
ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n{failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': analysis_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("ANALYSIS BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    if failed_count > 0:
        ux.warning(f"{failed_count} job(s) failed to submit")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Monitor job progress in Step_02 or proceed to Grouping stage")

except Exception as e:
    ux.error(f"Step completion failed: {str(e)}")
    step.fail(str(e))
    raise